In [1]:
import vk_api
import json
from pyvis.network import Network

In [2]:
login, password = 'your_login', 'your_password'

In [4]:
# получаем код аутентификации и доступ к api Вконтакте
vk_session = vk_api.VkApi(login, password)

try:
    vk_session.auth(token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)

vk = vk_session.get_api()

In [5]:
# запрашиваем список друзей пользователя с указанным id
# указываем какое-нибудь дополнительное поле, 
# чтобы вернулся список объектов с общедоступными атрибутами, а не список id
my_id = '43739454'
my_friends_dict = vk.friends.get(user_id=my_id, order='hints', fields='sex')

In [6]:
# порождаем объект Network для визуализации графа, адаптируем под Jupiter Notebook
graph_vis = Network(width=700, height=400, notebook=True)

# проходим по полученному списку друзей и добавляем каждый элемент в вершину графа
for item in my_friends_dict['items']:
    graph_vis.add_node(item.get('id'), item.get('first_name') + '\n' + item.get('last_name'))

my_friends_ids = [friend.get('id') for friend in my_friends_dict['items']]

# проходим по полученному списку друзей и запрашиваем друзей для каждого активного пользователя
# во вложенном цикле проверяем наличие "дргуа друга" в изначальном списке друзей и добавлем ребро графа
for friend in my_friends_dict['items']:
   if not friend.get('deactivated'):
        friend_friends_dict = vk.friends.get(user_id=friend.get('id'), order='hints', fields='sex')
        for subfriend in friend_friends_dict['items']:
            if subfriend.get('id') in my_friends_ids:
                graph_vis.add_edge(friend.get('id'), subfriend.get('id'))

In [7]:
# визуализируем интерактивный граф
# под графом отобразим панель настроек: можно покрутить и сохранить подходящую конфигурацию
graph_vis.show_buttons()
graph_vis.show('example.html')

In [ ]:
#на полученном графе довольно чётко выражены отдельные группы пользователей, соответствующие кругам общения